<div class="alert" style="background-color:#BBA14F; color:white; padding:0px 10px; border-radius:5px;">
    <h1 style='margin:15px 15px; color:#000000; font-size:32px'><b>LSTM</b></h1>
        <h2 style='margin:15px 15px; color:#000000; font-size:24px'>Human Activity Recognition Problem</h2>
</div>

The work is under the **"Master Thesis"** by **Chau Tran** with the supervision from **Prof. Roland Olsson**.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.util.tf_export import keras_export
from tensorflow.keras.backend import eval

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, RobustScaler

import tensorboard
import keras
from keras.utils import tf_utils
import pandas as pd #pd.plotting.register_matplotlib_converters
import numpy as np
import sys, os, math, time, datetime, re

print("tf: ", tf.__version__)
print("tb: ", tensorboard.__version__)
print(os.getcwd())

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(1)
tf.config.set_visible_devices([], 'GPU')
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'

snapshot = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Debugging with Tensorboard
logdir="logs/fit/lstm/" + snapshot
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# tf.debugging.experimental.enable_dump_debug_info(logdir, tensor_debug_mode="FULL_HEALTH", circular_buffer_size=-1)

ISMOORE_DATASETS = True
path = '../../../../Datasets/6_har/0_WISDM/WISDM_ar_v1.1/WISDM_ar_v1.1_processed/WISDM_ar_v1.1_wt_overlap'
fileslist = [f for f in sorted(os.listdir(path)) if os.path.isfile(os.path.join(path, f))]
FILESNUMBER = 1
LSTMNUMBER  = 1

with open("../../params/params_har.txt") as f:
    hyperparams = dict([re.sub('['+' ,\n'+']','',x.replace(' .', '')).split('=') for x in f][1:-1])
hyperparams = dict([k, float(v)] for k, v in hyperparams.items())
hyperparams['testSize'] = 0.500
hyperparams['noUnits'] = 81
print(hyperparams)

def seperateValues(data, noInput, noOutput, isMoore=True):
    x_data, y_data = None, None
    for i in range(data.shape[0]):
        if isMoore:
            x_data_i = data[i].reshape(-1, noInput+noOutput)
            x_data_i, y_data_i = x_data_i[:, 0:noInput], x_data_i[-1, noInput:]
        else:
            x_data_i = data[i][:-1].reshape(-1, noInput)
            y_data_i = data[i][-1].reshape(-1, noOutput)
        x_data = x_data_i[np.newaxis,:,:] if x_data is None else np.append(x_data, x_data_i[np.newaxis,:,:], axis=0)
        y_data = y_data_i.reshape(1, -1) if y_data is None else np.append(y_data, y_data_i.reshape(1, -1), axis=0)
    return x_data, y_data

def fromBit( b ) :
    return -0.9 if b == 0.0 else 0.9

class CustomMetricError(tf.keras.metrics.MeanMetricWrapper):
    def __init__(self, name='custom_metric_error', dtype=None, threshold=0.5):
        super(CustomMetricError, self).__init__(
            customMetricfn_tensor, name, dtype=dtype, threshold=threshold)

def customMetricfn_tensor(true, pred, threshold=0.5):
    true = tf.convert_to_tensor(true)
    pred = tf.convert_to_tensor(pred)
    threshold = tf.cast(threshold, pred.dtype)
    pred = tf.cast(pred >= threshold, pred.dtype)
    true = tf.cast(true >= threshold, true.dtype)
    return keras.backend.mean(tf.equal(true, pred), axis=-1)

def customMetricfn(y_true, y_pred):
    count, numCorrect = 0, 0
    for i in range( y_true.shape[0] ) :
        for j in range( y_pred.shape[ 1 ] ) :
            count += 1
            if isCorrect( y_true[ i, j ], y_pred[ i, j ] ) :
                numCorrect += 1
    return (numCorrect/count)

def isCorrect( target, actual ) :
    y1 = False if target < 0.0 else True
    y2 = False if actual < 0.0 else True
    return y1 == y2 

class customLRSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initialLearningRate, learningRateDecay, decayDurationFactor, numTrainingSteps, glorotScaleFactor=0.1, orthogonalScaleFactor=0.1, name=None):
        self.initialLearningRate = initialLearningRate
        self.learningRateDecay = learningRateDecay
        self.decayDurationFactor = decayDurationFactor
        self.glorotScaleFactor = glorotScaleFactor
        self.orthogonalScaleFactor = orthogonalScaleFactor
        self.numTrainingSteps = numTrainingSteps
        self.name = name
        self.T = tf.constant(self.decayDurationFactor * self.numTrainingSteps, dtype=tf.float32, name="T")
    
    def __call__(self, step):
        self.step = tf.cast(step, tf.float32)
        self.lr = tf.cond(self.step > self.T, 
                           lambda: tf.constant(self.learningRateDecay * self.initialLearningRate, dtype=tf.float32),
                           lambda: self.initialLearningRate * (1.0 - (1.0 - self.learningRateDecay) * self.step / self.T)
                          )
        return self.lr

def lstm_wLRS_wtCMF_model(noInput, noOutput, timestep):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=hyperparams['noUnits'], input_shape=[timestep, noInput],
                   activation='tanh', recurrent_activation='sigmoid', unroll=False, use_bias=True,
                   recurrent_dropout=0.0, return_sequences=False, name='LSTM_layer'))
    model.add(tf.keras.layers.Dense(noInput+noOutput, activation='tanh', name='MLP_layer'))
    model.add(tf.keras.layers.Dense(noOutput))
    optimizer = tf.keras.optimizers.Adam(learning_rate=customLRSchedule(hyperparams['initialLearningRate'], hyperparams['learningRateDecay'], hyperparams['decayDurationFactor'], hyperparams['numTrainingSteps']), \
                                    beta_1=hyperparams['beta1'], beta_2=hyperparams['beta2'], epsilon=hyperparams['epsilon'], amsgrad=False, name="tunedAdam_lstm")
    model.compile(optimizer=optimizer, loss = 'mse', run_eagerly=False)
    return model

tf:  2.7.0
tb:  2.7.0
C:\Users\chaut\OneDrive - Heriot-Watt University\HIOF_Master\Master_Thesis\NewLSTM\Codes\tf_implementations\notebooks\rnnplus_for_har
{'batchSize': 4.0, 'numTrainingSteps': 40000.0, 'beta1': 0.974833, 'beta2': 0.99689, 'epsilon': 0.00388, 'decayDurationFactor': 0.979079, 'initialLearningRate': 0.002798, 'learningRateDecay': 0.001025, 'glorotScaleFactor': 0.1, 'orthogonalScaleFactor': 0.1, 'testSize': 0.5, 'noUnits': 81}


In [2]:
print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
df_train = np.array(pd.read_csv(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=50.train.csv'), skiprows=1))
df_val = np.array(pd.read_csv(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=50.val.csv'), skiprows=1))

print(df_train.shape, df_val.shape)

with open(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=50.train.csv'), "r") as fp:
    [noIn, noOut] = [int(x) for x in fp.readline().replace('\n', '').split(',')]
        
print('Step 2: Separating values and labels.')
x_train, y_train = seperateValues(df_train, noIn, noOut, isMoore=ISMOORE_DATASETS)
x_val, y_val = seperateValues(df_val, noIn, noOut, isMoore=ISMOORE_DATASETS)
for i in range( x_train.shape[ 0 ] ) :
    for j in range( x_train.shape[ 1 ] ) :
        for k in range( x_train.shape[ 2 ] ) :
            x_train[ i, j, k ] = fromBit( x_train[ i, j, k ] )

for i in range( y_train.shape[ 0 ] ) :
    for j in range( y_train.shape[ 1 ] ) :
        y_train[ i, j ] = fromBit( y_train[ i, j ] )

for i in range( x_val.shape[ 0 ] ) :
    for j in range( x_val.shape[ 1 ] ) :
        for k in range( x_val.shape[ 2 ] ) :
            x_val[ i, j, k ] = fromBit( x_val[ i, j, k ] )

for i in range( y_val.shape[ 0 ] ) :
    for j in range( y_val.shape[ 1 ] ) :
        y_val[ i, j ] = fromBit( y_val[ i, j ] )

print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)

model = lstm_wLRS_wtCMF_model(noIn, noOut, timestep=40)
model_history = model.fit(
                    x_train, y_train,
                    batch_size=int(hyperparams['batchSize']),
                    verbose=1, # Suppress chatty output; use Tensorboard instead
                    # epochs=10,
                    epochs=int(hyperparams['numTrainingSteps']/(x_train.shape[0])),
                    validation_data=(x_val, y_val),
                    shuffle=True,
                    use_multiprocessing=False,
                    callbacks=[tensorboard_callback]
                )
y_pred = model.predict(x_val, verbose=1, batch_size=int(hyperparams['batchSize']))
# result_tf[model_name][filename].append(round(customMetricfn(y_val, y_pred), 5)*100)

val_performance = model.evaluate(x_val, y_val, batch_size=1, verbose=1)
y_pred = model.predict(x_val, verbose=1)

print(round(customMetricfn(y_val, y_pred), 5))

Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(12535, 360) (12537, 360)
Step 2: Separating values and labels.
+ Training set:    (12535, 40, 3) (12535, 6) float64
+ Validating set:  (12537, 40, 3) (12537, 6) float64
Epoch 1/3
3134/3134 [==============================] - 41s 13ms/step - loss: 0.3617 - val_loss: 0.3423
Epoch 2/3
3134/3134 [==============================] - 44s 14ms/step - loss: 0.3473 - val_loss: 0.3534
Epoch 3/3
392/392 [==============================] - 3s 8ms/step
0.84599


In [3]:
print('Step 1: Dividing the training and testing set with ratio 1:1 (50%).')
df_train = np.array(pd.read_csv(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=0.all.shuffled.train.csv'), skiprows=0))
df_val = np.array(pd.read_csv(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=0.all.shuffled.test.csv'), skiprows=0))

print(df_train.shape, df_val.shape)

# with open(os.path.join(path,'wisdm.ni=3.no=6.ts=40.os=40.spit=0.all.shuffled.train.csv'), "r") as fp:
#     [noIn, noOut] = [int(x) for x in fp.readline().replace('\n', '').split(',')]
noIn, noOut = 3, 6

print('Step 2: Separating values and labels.')
x_train, y_train = seperateValues(df_train, noIn, noOut, isMoore=ISMOORE_DATASETS)
x_val, y_val = seperateValues(df_val, noIn, noOut, isMoore=ISMOORE_DATASETS)
for i in range( x_train.shape[ 0 ] ) :
    for j in range( x_train.shape[ 1 ] ) :
        for k in range( x_train.shape[ 2 ] ) :
            x_train[ i, j, k ] = fromBit( x_train[ i, j, k ] )

for i in range( y_train.shape[ 0 ] ) :
    for j in range( y_train.shape[ 1 ] ) :
        y_train[ i, j ] = fromBit( y_train[ i, j ] )

for i in range( x_val.shape[ 0 ] ) :
    for j in range( x_val.shape[ 1 ] ) :
        for k in range( x_val.shape[ 2 ] ) :
            x_val[ i, j, k ] = fromBit( x_val[ i, j, k ] )

for i in range( y_val.shape[ 0 ] ) :
    for j in range( y_val.shape[ 1 ] ) :
        y_val[ i, j ] = fromBit( y_val[ i, j ] )

print("+ Training set:   ", x_train.shape, y_train.shape, x_train.dtype)
print("+ Validating set: ", x_val.shape, y_val.shape, x_val.dtype)

model = lstm_wLRS_wtCMF_model(noIn, noOut, timestep=40)
model_history = model.fit(
                    x_train, y_train,
                    batch_size=int(hyperparams['batchSize']),
                    verbose=1, # Suppress chatty output; use Tensorboard instead
                    # epochs=10,
                    epochs=int(hyperparams['numTrainingSteps']/(x_train.shape[0])),
                    validation_data=(x_val, y_val),
                    shuffle=True,
                    use_multiprocessing=False,
                    callbacks=[tensorboard_callback]
                )
y_pred = model.predict(x_val, verbose=1, batch_size=int(hyperparams['batchSize']))
# result_tf[model_name][filename].append(round(customMetricfn(y_val, y_pred), 5)*100)

val_performance = model.evaluate(x_val, y_val, batch_size=int(hyperparams['batchSize']), verbose=1)
y_pred = model.predict(x_val, verbose=1)

print(round(customMetricfn(y_val, y_pred), 5))

Step 1: Dividing the training and testing set with ratio 1:1 (50%).
(12506, 360) (12505, 360)
Step 2: Separating values and labels.
+ Training set:    (12506, 40, 3) (12506, 6) float64
+ Validating set:  (12505, 40, 3) (12505, 6) float64
Epoch 1/3
3127/3127 [==============================] - 44s 14ms/step - loss: 0.3643 - val_loss: 0.3527
Epoch 2/3
3127/3127 [==============================] - 40s 13ms/step - loss: 0.3461 - val_loss: 0.3445
Epoch 3/3
391/391 [==============================] - 5s 12ms/step
0.84626
